In [75]:
from itertools import combinations
import math

In [76]:
def hamming_distance(v1, v2):
    """Вычисляет расстояние Хемминга между двумя бинарными векторами."""
    return sum(a != b for a, b in zip(v1, v2))

def hamming_ball(center, radius, n):
    """Возвращает множество всех векторов в F_2^n, находящихся на расстоянии <= radius от center."""
    ball = []
    for i in range(n + 1):
        if i <= radius:
            # Все векторы с весом i, отличающиеся от center в i позициях
            for positions in combinations(range(n), i):
                vec = list(center)
                for pos in positions:
                    vec[pos] = 1 - vec[pos]  # инвертируем бит
                ball.append(tuple(vec))
    return set(ball)

# def volume_of_hamming_ball(n, t):
#     """Объём шара Хемминга радиуса t в F_2^n."""
#     vol = 0
#     for i in range(t + 1):
#         # Число способов выбрать i позиций из n
#         c = 1
#         for j in range(i):
#             c = c * (n - j) // (j + 1)
#         vol += c
#     return vol

# def volume_of_hamming_ball(n, t):
#     """Объём шара Хемминга радиуса t в F_2^n."""
#     volume = 0
#     for k in range(t + 1):
#         if k > n:
#             break
#         # Вычисляем биномиальный коэффициент C(n, k)
#         if k == 0 or k == n:
#             volume += 1
#         else:
#             # Используем комбинаторную формулу с целочисленной арифметикой
#             coeff = math.comb(n, k)
#             volume += coeff
    
#     return volume

def volume_of_hamming_ball(n, t):
    """Объём шара Хемминга радиуса t в F_2^n."""
    result = 0
    # Суммируем все биномиальные коэффициенты от 0 до t
    for k in range(t + 1):
        if k > n:  # Если k больше n, дальше не считаем
            break
        
        # Считаем C(n, k) самым простым способом
        # C(n, k) = n! / (k! * (n-k)!)
        
        # Считаем факториалы в лоб
        fact_n = 1
        for i in range(1, n + 1):
            fact_n *= i
        
        fact_k = 1
        for i in range(1, k + 1):
            fact_k *= i
        
        fact_nk = 1
        for i in range(1, n - k + 1):
            fact_nk *= i
        
        # Биномиальный коэффициент
        comb = fact_n // (fact_k * fact_nk)
        
        # Добавляем к результату
        result += comb
    
    return result


def is_perfect_code(n, t):
    """
    Проверяет, существует ли совершенный код (разбиение пространства на непересекающиеся шары радиуса t).
    Возвращает:
      - True/False
      - Список центров (если удалось построить или известен)
    """
    total_vectors = 2 ** n
    ball_size = volume_of_hamming_ball(n, t)
    print(f"{total_vectors=}, {ball_size=}")

    if total_vectors % ball_size != 0:
        return False, []

    # 1. Тривиальный случай: t == 0 -> каждый вектор — центр
    if t == 0:
        centers = [tuple([0]*n)]
        return True, centers

    # 2. Тривиальный случай: t >= n -> вся сфера = всё пространство
    if t >= n:
        centers = [tuple([0]*n)]
        return True, centers

    # 3. Повторные коды нечётной длины: t = (n-1)/2, n нечётно, |C| = 2
    if n % 2 == 1 and t == (n - 1) // 2:
        centers = [tuple([0]*n), tuple([1]*n)]
        return True, centers

    # 4. Коды Хэмминга: t=1, n = 2^r - 1, r>=2
    if t == 1:
        r = 0
        while (1 << r) - 1 < n:
            r += 1
        if (1 << r) - 1 == n and r >= 2:
            return True, [tuple([0]*n)]

    # 5. Бинарный код Голея: t=3, n=23
    if t == 3 and n == 23:
        return True, [tuple([0]*23)]
    return False, []



In [77]:
def main(n: int, t: int):
    print("=== Проверка возможности разбиения F_2^n на шары Хемминга радиуса t ===")
    print(f"{n=} {t=}")

    exists, centers = is_perfect_code(n, t)
    vol = volume_of_hamming_ball(n, t)
    total = 2 ** n

    if not exists:
        print(f"Разбиение невозможно для n={n}, t={t}.")
        return

    expected_centers_count = total // vol

    centers_complete = (len(centers) == expected_centers_count)

    if centers_complete:
        print(f"Разбиение возможно. Всего центров: {len(centers)}")
        print("Координаты всех центров:")
        for i, c in enumerate(centers, 1):
            print(f"{i}: {''.join(map(str, c))}")

        print("\n" + "="*50)

        for idx, center in enumerate(centers, 1):
            print(f"\nЦентр {idx}: {''.join(map(str, center))}")
            ball = hamming_ball(center, t, n)
            print("Векторы в шаре:")
            for vec in sorted(ball):
                print(''.join(map(str, vec)))
    else:
        print("Разбиение существует, но предоставлен неполный список центров.")
        print(f"Ожидается {expected_centers_count} центров, получено {len(centers)}.")
        if centers:
            print("Известные центры:")
            for i, c in enumerate(centers, 1):
                print(f"{i}: {''.join(map(str, c))}")
            print("Пример шара вокруг первого центра:")
            center = centers[0]
            ball = hamming_ball(center, t, n)
            print(f"Центр: {''.join(map(str, center))}")
            print(f"Векторы в шаре (количество: {len(ball)}):")
            for vec in sorted(ball):
                print(''.join(map(str, vec)))

In [78]:
main(
    n=3,
    t=1,
)

=== Проверка возможности разбиения F_2^n на шары Хемминга радиуса t ===
n=3 t=1
total_vectors=8, ball_size=4
Разбиение возможно. Всего центров: 2
Координаты всех центров:
1: 000
2: 111


Центр 1: 000
Векторы в шаре:
000
001
010
100

Центр 2: 111
Векторы в шаре:
011
101
110
111
